# Version information

In [11]:
from datetime import date
print("Running date:", date.today().strftime("%B %d, %Y"))
import pyleecan
print("Pyleecan version:" + pyleecan.__version__)
import SciDataTool
print("SciDataTool version:" + SciDataTool.__version__)

Running date: March 05, 2022
Pyleecan version:1.3.7
SciDataTool version:1.4.24


# How to compute magnetic forces using Force Module

This tutorial shows the different steps to **compute magnetic forces** with pyleecan.
 
The notebook related to this tutorial is available on [GitHub](https://github.com/Eomys/pyleecan/tree/master/Tutorials/tuto_Force.ipynb).

To demonstrate the capabilities and the use of the SciDataTool objects, a simulation is launched with FEMM, with imposed currents, using periodicity and parallelization to reduce execution time.

In [ ]:
from numpy import exp, sqrt, pi
from os.path import join
from pyleecan.Classes.Simu1 import Simu1
from pyleecan.Classes.InputCurrent import InputCurrent
from pyleecan.Classes.MagFEMM import MagFEMM
from pyleecan.Classes.ForceMT import ForceMT
from pyleecan.Classes.ForceTensor import ForceTensor
from pyleecan.Classes.Output import Output
from pyleecan.Functions.load import load
from pyleecan.definitions import DATA_DIR
from pyleecan.Classes.OPdq import OPdq
from multiprocessing import cpu_count

# Load the machine
Toyota_Prius = load(join(DATA_DIR, "Machine", "Toyota_Prius.json"))

# Simulation initialization
simu = Simu1(name="FEMM_periodicity", machine=Toyota_Prius)


simu.input = InputCurrent(
#    Na_tot=252 * 8,
#    Nt_tot=50 * 8,
    Na_tot=5 * 2 ** 8, Nt_tot=2
)
# Set Id/Iq according to I0/Phi0
simu.input.OP = OPdq(N0=1000)
simu.input.OP.set_I0_Phi0(I0=250 / sqrt(2), Phi0=140*pi/180)

# Definition of the magnetic simulation: with periodicity
simu.mag = MagFEMM(is_periodicity_a=True, is_periodicity_t=True, nb_worker=cpu_count(),is_get_meshsolution=True)
simu.force = ForceMT(is_periodicity_a=True, is_periodicity_t=True)

simu_sym = Simu1(init_dict=simu.as_dict())
simu_sym.mag.is_periodicity_a = True

out = Output(simu=simu_sym)
simu_sym.run()
save_plot_path="D:/KDH/pyleecan_out_figure"

out.mag.meshsolution.plot_mesh(
    save_path=join(save_plot_path, simu.name + "_mesh.png"), is_show_fig=False
)

out.mag.meshsolution.plot_mesh(
    group_names="stator core",
    save_path=join(save_plot_path, simu.name + "_mesh_stator.png"),
    is_show_fig=False,
)

# Run simulations
out = simu.run()

#실행안됨

In [ ]:
##backup 실행안됨
import numpy as np
from SciDataTool import DataTime, VectorField, Data1D

from pyleecan.Classes.Interpolation import Interpolation
from pyleecan.Classes.MeshSolution import MeshSolution
from pyleecan.Classes.SolutionVector import SolutionVector

dim = 2
Time = simu.axes_dict["time"]
Nt_tot = Time.get_length()  # Number of time step


meshsolution_group = meshsolution_mag.get_group(meshsolution_mag.group)
mesh = meshsolution_group.get_mesh()

# New meshsolution object for output, that could be different from the one inputed
meshsolution = MeshSolution(mesh=[mesh.copy()], is_same_mesh=True, dimension=dim)

# Load magnetic flux B and H and mu objects
B_sol = meshsolution_group.get_solution(label="B")
H_sol = meshsolution_group.get_solution(label="H")
mu_sol = meshsolution_group.get_solution(label="\mu")

# Import time vector from Time Data object
if self.is_periodicity_t is not None:
    is_periodicity_t = self.is_periodicity_t

is_periodicity_t, is_antiper_t = Time.get_periodicity()
time = Time.get_values(
    is_oneperiod=is_periodicity_t,
    is_antiperiod=is_antiper_t and is_periodicity_t,
)

# Load magnetic flux B and H of size (Nt_tot, nb_elem, dim) and mu (Nt_tot, nb_elem)
resultB = B_sol.field.get_xyz_along(
    "indice",
    "time=axis_data",
    axis_data={"time": time},
    is_squeeze=False,
)
indice = resultB["indice"]  # Store elements indices

Bx = resultB["comp_x"]
By = resultB["comp_y"]
B = np.stack((Bx, By), axis=2)

resultH = H_sol.field.get_xyz_along(
    "indice",
    "time=axis_data",
    axis_data={"time": time},
    is_squeeze=False,
)
Hx = resultH["comp_x"]
Hy = resultH["comp_y"]
H = np.stack((Hx, Hy), axis=2)

resultmu = mu_sol.field.get_along(
    "indice",
    "time=axis_data",
    axis_data={"time": time},
    is_squeeze=False,
)
mu = resultmu["\\mu"]

# Move time axis at the end for clarity purpose
B = np.moveaxis(B, 0, -1)
H = np.moveaxis(H, 0, -1)
mu = np.moveaxis(mu, 0, -1)

# Loop on elements and nodes for nodal forces
f, connect = self.element_loop(mesh, B, H, mu, indice, dim, Nt_tot)

실행안됨

In [6]:
from pyleecan.Classes.ForceTensor import ForceTensor
#simu.force = ForceMT(is_periodicity_a=True, is_periodicity_t=True)

out = Simu1(name="FEMM_periodicity", machine=Toyota_Prius)
out.input = InputCurrent(
#    Na_tot=252 * 8,
#    Nt_tot=50 * 8,
    Na_tot=5 * 2 ** 8, Nt_tot=2
)
# Set Id/Iq according to I0/Phi0
out.input.OP = OPdq(N0=1000)
out.input.OP.set_I0_Phi0(I0=250 / sqrt(2), Phi0=140*pi/180)

# Definition of the magnetic simulation: with periodicity
out.mag = MagFEMM(is_periodicity_a=True, is_periodicity_t=True, nb_worker=cpu_count(), is_get_meshsolution=True)
out.force=ForceTensor(
        is_periodicity_a=False,
        is_periodicity_t=False,
        tensor={
            "magnetostriction": False,
        },
    )
out.run()

# Run simulations



NameError: name 'Toyota_Prius' is not defined

실행확인 ForceTensor

In [12]:
import pytest

from os.path import join
from numpy import zeros, exp, pi, real, meshgrid, mean
from multiprocessing import cpu_count
from numpy.testing import assert_array_almost_equal
from SciDataTool import DataTime, VectorField, Data1D
import csv
import numpy as np
import matplotlib.pyplot as plt

from pyleecan.Classes.ForceTensor import ForceTensor
from pyleecan.Classes.OPdq import OPdq
from pyleecan.Classes.Simu1 import Simu1
from pyleecan.Classes.MagFEMM import MagFEMM
from pyleecan.Classes.InputCurrent import InputCurrent
from pyleecan.Classes.SolutionVector import SolutionVector

from pyleecan.Functions.load import load
from pyleecan.definitions import DATA_DIR
save_plot_path="D:/KDH/pyleecan_out_figure"
save_path= save_plot_path
DELTA = 1e-6



# Load machine
Benchmark = load(join(DATA_DIR, "Machine", "Benchmark.json"))
# Prepare simulation
simu = Simu1(name="Benchmark_Tensor", machine=Benchmark)

simu.input = InputCurrent(
    OP=OPdq(N0=1200, Id_ref=0, Iq_ref=0),
    Ir=None,
    Na_tot=2 ** 6,
    Nt_tot=1,
)

simu.elec = None

simu.mag = MagFEMM(
    type_BH_stator=1,  # 0 for saturated
    type_BH_rotor=1,
    is_periodicity_a=False,
    is_periodicity_t=False,
    is_get_meshsolution=True,
    is_sliding_band=False,
    # nb_worker=cpu_count(),
    Kmesh_fineness=1,
)
simu.force = ForceTensor(
    is_periodicity_a=False,
    is_periodicity_t=False,
    tensor={
        "magnetostriction": True,
    },
)

# Run simulation
out = simu.run()

# CSV import



[15:25:20] Starting running simulation Benchmark_Tensor (machine=Benchmark)
[15:25:20] Starting Magnetic module
[15:25:22] Computing Airgap Flux in FEMM
[15:25:24] Starting Force module
[15:25:28] End of simulation Benchmark_Tensor


In [26]:
path = save_path.replace("pyleecan_out_figure", "pyleecan_out_figure/Result/Plot/Data/Benchmark_model_stator_ms.csv")
open(path,'r')

<_io.TextIOWrapper name='D:/KDH/pyleecan_out_figure/Result/Plot/Data/Benchmark_model_stator_ms.csv' mode='r' encoding='UTF-8'>

Plot까지 됨 뭘 했는지는 잘모르겠음. csv는 아니고 해석한 out.force plot임

In [22]:
reader = csv.reader(path, skipinitialspace=True)
save_path

out.force.meshsolution.plot_glyph(label="my_vector", is_point_arrow=False,factor=1/200)

In [24]:
path = save_path.replace("pyleecan_out_figure", "pyleecan_out_figure/Result/Plot/Data/Benchmark_model_stator_ms.csv")
with open(path, "r") as file:
    reader = csv.reader(file, skipinitialspace=True)
    l1 = next(reader)
    l2 = next(reader)
    l3 = next(reader)
    nb_node = int(l2[1])
    dim = int(l2[2])
    Nt_tot = int(l2[3])
    f2 = np.zeros((nb_node, dim))
    node_number_list = []
    indices_nodes = []
    for row in reader:
        f2[int(row[0])][0] = 1000 * float(row[3])
        f2[int(row[0])][1] = 1000 * float(row[4])
        node_number_list.append(int(row[10]))
        indices_nodes.append(int(row[11]))




# Little trick to reshape f2 so that it can be compared to f, since Indices_Points2 isn't taken into account yet
_, _, connectivity = np.intersect1d(
    indices_nodes, node_number_list, return_indices=True
)
f2 = f2[connectivity, :]

f2 = f2.reshape((Nt_tot, nb_node, dim))

components2 = {}
Indices_Point2 = Data1D(
    name="indice", values=np.array(node_number_list), is_components=True
)

Time = out.mag.meshsolution.solution[0].field.get_axes()[0]

fx2_data = DataTime(
    name="Nodal force 2 (x)",
    unit="N",
    symbol="Fx2",
    axes=[Time, Indices_Point2],
    values=f2[..., 0],
)
components2["comp_x"] = fx2_data

fy2_data = DataTime(
    name="Nodal force 2 (y)",
    unit="N",
    symbol="Fy2",
    axes=[Time, Indices_Point2],
    values=f2[..., 1],
)
components2["comp_y"] = fy2_data

vec_force2 = VectorField(name="Nodal forces 2", symbol="F2", components=components2)
solforce2 = SolutionVector(field=vec_force2, type_cell="node", label="F2")
out.force.meshsolution.solution.append(solforce2)

out.force.meshsolution.plot_glyph(
    label="F",
    is_point_arrow=True,
    # is_show_fig=True,
    save_path=join(save_path, "magneto_plot_glyph.png"),
)

out.force.meshsolution.plot_glyph(
    label="F",
    is_point_arrow=True,
    # is_show_fig=True,
    save_path=join(save_path, "magneto_plot_glyph2.png"),
)

# Comparisons

computed_forces_x = (
    out.force.meshsolution.solution[0].field.components["comp_x"].values
)
reference_forces_x = f2[..., 0]
computed_forces_y = (
    out.force.meshsolution.solution[0].field.components["comp_y"].values
)
reference_forces_y = f2[..., 1]

relevant_mask_x = reference_forces_x > 1e-3 * np.max(reference_forces_x)
relevant_mask_y = reference_forces_y > 1e-3 * np.max(reference_forces_y)

relevant_computed_forces_x = computed_forces_x[relevant_mask_x]
relevant_reference_forces_x = reference_forces_x[relevant_mask_x]
relevant_computed_forces_y = computed_forces_y[relevant_mask_y]
relevant_reference_forces_y = reference_forces_y[relevant_mask_y]

big_diff_mask_x = np.logical_and(
    np.abs(computed_forces_x - reference_forces_x) / np.abs(reference_forces_x)
    > 0.2,
    np.abs(computed_forces_x - reference_forces_x) / np.abs(reference_forces_x)
    < 1.5,
)
big_diff_mask_y = np.logical_and(
    np.abs(computed_forces_y - reference_forces_y) / np.abs(reference_forces_y)
    > 0.2,
    np.abs(computed_forces_y - reference_forces_y) / np.abs(reference_forces_y)
    < 1.5,
)

big_diff_x = np.where(
    big_diff_mask_x,
    computed_forces_x - reference_forces_x,
    np.zeros(computed_forces_x.shape),
)
big_diff_y_x = np.where(
    big_diff_mask_x,
    computed_forces_y - reference_forces_y,
    np.zeros(computed_forces_y.shape),
)

components_diff_x = {}
fx_diff_data = DataTime(
    name="Nodal force diff x (x)",
    unit="N",
    symbol="Fxdx",
    axes=[Time, Indices_Point2],
    values=big_diff_x,
)
components_diff_x["comp_x"] = fx_diff_data

fy_diff_data = DataTime(
    name="Nodal force diff x (y)",
    unit="N",
    symbol="Fydx",
    axes=[Time, Indices_Point2],
    values=big_diff_y_x,
)
components_diff_x["comp_y"] = fx_diff_data

vec_force_dx = VectorField(
    name="Nodal forces dx", symbol="Fdx", components=components_diff_x
)
solforce_dx = SolutionVector(field=vec_force_dx, type_cell="node", label="Fdx")
out.force.meshsolution.solution.append(solforce_dx)

out.force.meshsolution.plot_glyph(
    label="Fdx",
    is_point_arrow=True,
    # is_show_fig=True,
    save_path=join(save_path, "magneto_plot_glyph2.png"),
)



C:\Users\KDH2018-PC\anaconda3\envs\py38_openturn\lib\site-packages\pyvista\core\dataset.py:459: PyvistaDeprecationWarning: Use of `DataSet.vectors` to add vector data is deprecated. Use `DataSet['vector_name'] = data`. Use `DataSet.active_vectors_name = 'vector_name' to make active.
  warnings.warn(


IndexError: boolean index did not match indexed array along dimension 1; dimension is 3897 but corresponding boolean dimension is 3748

## Force Module
The Force abstract class will make it possible to define different ways of calculating forces. 

The ForceMT class inherits from Force class. ForceMT is dedicated to the computation of air-gap surface force based on the Maxwell stress tensor \[[source](https://eomys.com/IMG/pdf/comparison-main-magnetic.pdf)\]. 

Here, we get the results from a magnetic simulation without any force calculation. The Force module is initialized and run alone. 

In [ ]:
from pyleecan.Classes.Simu1 import Simu1
from pyleecan.Classes.ForceMT import ForceMT

# Create the Simulation
mySimu = Simu1(name="Tuto_Force")  
mySimu.parent = out
mySimu.force = ForceMT()

# Run only the force module


Once the simulation is finished, the results are stored in the force part of the output (i.e. _myResults.force_ ) and we can call different plots. This object contains:   
- *Time*: Time axe
- *Angle*: Angular position axe   
- *AGSF*: Airgap surface force (Radial and Tangential component)
    
**Output** object embbed different plot to visualize results easily. You can find a dedicated tutorial [here](https://www.pyleecan.org/tuto_Plots.html).

Here are some example of useful plots.

In [13]:
from pyleecan.Functions.Plot import dict_2D, dict_3D
out.force.AGSF.plot_2D_Data("angle{°}", **dict_2D)
out.force.AGSF.plot_2D_Data("wavenumber=[0,78]", **dict_2D)

In [14]:
from numpy import pi

#------------------------------------------------------
# Plot the air-gap force as a function of time with the time fft
out.force.AGSF.plot_2D_Data("time","angle[10]", is_auto_ticks=False, **dict_2D)
out.force.AGSF.plot_2D_Data("freqs=[0,4000]", is_auto_ticks=False, **dict_2D)
#------------------------------------------------------

The following plot displays the radial air-gap surface force over time and angle. 

In [15]:
#------------------------------------------------------
# Plot the tangential force as a function of time and space
out.force.AGSF.plot_3D_Data("time", "angle{°}", is_2D_view=True, **dict_3D)
out.force.AGSF.plot_3D_Data("time", "angle{°}", is_2D_view=False, **dict_3D)
#------------------------------------------------------